##### DATA

In [ ]:
df_data.columns

Index(['PM10', 'PM25', 'O3', 'NO2', 'SO2', 'tem', 'rain', 'hum', 'vapor',
       'dew', 'press', 'cloud', 'wind_x', 'wind_y', 'ammonia', 'toluen',
       'methyle', 'benzene'],
      dtype='object')

In [ ]:
df_data = df_data.interpolate(method = 'linear')

In [ ]:
# Train, Vlidation, test set
air_train = df_data[df_data.index.astype(str).str[:4] == '2019']
air_valid = df_data[df_data.index.astype(str).str[:4] == '2020']
air_test = df_data[df_data.index.astype(str).str[:4] == '2021']

air_train.shape, air_valid.shape, air_test.shape

((8760, 18), (8784, 18), (8760, 18))

In [ ]:
air_train.isnull().sum()

PM10       0
PM25       0
O3         0
NO2        0
SO2        0
tem        0
rain       0
hum        0
vapor      0
dew        0
press      0
cloud      0
wind_x     0
wind_y     0
ammonia    0
toluen     0
methyle    0
benzene    0
dtype: int64

In [ ]:
# # 1) 신뢰구간 이용
# def outlier(df, col, z):
#     return df[abs(df[col] - np.mean(df[col]))/np.std(df[col])>z].index
# # 2) IQR (25% 에서 75% 사이의 값 이용하기)
# def get_outlier(df=None, column=None, weight = 1.5):
#     quantile_25 = np.percentile(df[column].values, 25)
#     quantile_75 = np.percentile(df[column].values, 75)
    
#     IQR = quantile_75 - quantile_25
#     IQR_weight = IQR*weight
    
#     lowest = quantile_25 - IQR_weight
#     highest = quantile_75 + IQR_weight
    
#     outlier_idx = df[column][(df[column] < lowest) | (df[column] > highest)].index
#     return outlier_idx

# # !!!! outlier 구할 때 해당 column에 빈 값이 있으면 결과가 나오지 않음 dropna 이후에 진행하기.

In [ ]:
#def get_outlier(df=None, column=None, weight = 1.5):
# outlier_IQR = get_outlier(df = air_test, column='PM25', weight=1.5)
# len(outlier_IQR)

# air_test = air_test.drop(outlier_IQR)
# air_test = air_test.reset_index()
# air_test = air_test.drop('index', axis=1)

In [ ]:
air_train.columns

Index(['PM10', 'PM25', 'O3', 'NO2', 'SO2', 'tem', 'rain', 'hum', 'vapor',
       'dew', 'press', 'cloud', 'wind_x', 'wind_y', 'ammonia', 'toluen',
       'methyle', 'benzene'],
      dtype='object')

In [ ]:
y_train = air_train[['PM25']].reset_index()
x_train = air_train.copy()
y_valid = air_valid[['PM25']].reset_index()
x_valid = air_valid.copy()
y_test = air_test[['PM25']].reset_index()
x_test = air_test.copy()

In [ ]:
#StandardScaler & MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 

scaler.fit(x_train)

train_scaled = pd.DataFrame(scaler.transform(x_train))
valid_scaled = pd.DataFrame(scaler.transform(x_valid))
test_scaled = pd.DataFrame(scaler.transform(x_test))

train_scaled.columns = x_train.columns
valid_scaled.columns = x_valid.columns
test_scaled.columns = x_test.columns

train_scaled['predict_pm25'] = y_train['PM25']
valid_scaled['predict_pm25'] = y_valid['PM25']
test_scaled['predict_pm25'] = y_test['PM25']

In [ ]:
train_scaled

,PM10,PM25,O3,NO2,SO2,tem,rain,hum,vapor,dew,press,cloud,wind_x,wind_y,ammonia,toluen,methyle,benzene,predict_pm25
0,-0.062165,0.054807,-1.054984,0.486389,-0.855595,-1.709615,-0.146793,-1.774883,-1.308293,-2.066982,2.044835,-0.506958,0.670921,-1.718463,-0.143773,-0.313749,-0.314586,-0.174599,28.0
1,-0.323079,-0.273498,-0.883125,0.080660,-0.855595,-1.847492,-0.146793,-1.774883,-1.330662,-2.145097,2.085405,-1.260044,0.609534,-1.573274,-0.185917,-0.342465,-0.314586,-0.174599,23.0
2,-0.410050,-0.339159,-0.883125,0.080660,-0.855595,-1.985369,-0.146793,-1.727920,-1.330662,-2.188494,2.017788,-1.260044,1.557177,-1.047619,-0.214012,-0.356823,-0.314586,-0.174599,22.0
3,-0.323079,-0.076515,-0.997698,0.242952,-0.855595,-2.085643,-0.146793,-1.633992,-1.330662,-2.171135,2.044835,-1.260044,1.432994,-0.319824,-0.244449,-0.356823,-0.314586,-0.174599,26.0
4,-0.018680,0.120469,-0.825839,0.080660,-0.855595,-2.123246,-0.146793,-1.493101,-1.319477,-2.093020,2.004265,-1.260044,1.854645,-0.451935,-0.229231,-0.500405,-0.314586,-0.174599,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,-0.931876,-0.995769,-0.424834,-0.162778,-0.408785,-2.060575,-0.146793,-1.915775,-1.364217,-2.405479,1.990742,-1.260044,0.164476,-0.520660,0.410020,-0.643987,-0.314586,1.190381,12.0
8756,-1.105818,-1.061430,-0.310262,-0.487362,-0.408785,-2.098178,-0.146793,-1.962739,-1.375402,-2.474914,1.950172,-1.260044,-0.127114,0.709756,0.454958,-0.643987,-0.314586,1.074212,11.0
8757,-0.670963,-0.733125,-0.138403,-0.730800,-0.408785,-2.148315,-0.146793,-1.962739,-1.375402,-2.500952,1.909603,-1.260044,1.137838,-0.227347,0.499895,-0.629629,-0.314586,1.016128,16.0
8758,-1.192789,-0.930108,-0.252975,-0.649654,-0.408785,-2.210986,-0.146793,-1.821847,-1.364217,-2.405479,1.923126,-1.260044,0.388485,-0.203445,0.544833,-0.643987,-0.314586,0.929002,13.0


In [ ]:
x_train.columns

Index(['PM10', 'PM25', 'O3', 'NO2', 'SO2', 'tem', 'rain', 'hum', 'vapor',
       'dew', 'press', 'cloud', 'wind_x', 'wind_y', 'ammonia', 'toluen',
       'methyle', 'benzene'],
      dtype='object')

In [ ]:
train_x = train_scaled[['PM25', 'PM10', 'O3', 'NO2', 'SO2', 'tem', 'rain', 'wind_x', 'wind_y', 'hum',
       'press', 'cloud', 'ammonia', 'toluen', 'methyle']]
train_y = train_scaled[['predict_pm25']]

valid_x = valid_scaled[['PM25', 'PM10', 'O3', 'NO2', 'SO2', 'tem', 'rain', 'wind_x', 'wind_y', 'hum',
       'press', 'cloud', 'ammonia', 'toluen', 'methyle']]
valid_y = valid_scaled[['predict_pm25']]

test_x = test_scaled[['PM25', 'PM10', 'O3', 'NO2', 'SO2', 'tem', 'rain', 'wind_x', 'wind_y', 'hum',
       'press', 'cloud', 'ammonia', 'toluen', 'methyle']]
test_y = test_scaled[['predict_pm25']]

In [ ]:
train_x.shape, valid_x.shape, test_x.shape

((8760, 15), (8784, 15), (8760, 15))

In [ ]:
train_set = train_x.values
valid_set = valid_x.values
test_set = test_x.values

train_y = train_y.values
valid_y = valid_y.values
test_y = test_y.values

In [ ]:
# data change for LSTM

def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index, 24):
    indices = range(i-history_size, i)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 3000
past_history = 24
future_target = 24

x_train_multi, y_train_multi = multivariate_data(train_set, train_y, 0,
                                                       None, past_history,
                                                       future_target,
                                                       single_step=False)
x_val_multi, y_val_multi = multivariate_data(valid_set, valid_y,
                                                   0, None, past_history,
                                                   future_target,
                                                   single_step=False)
x_test_multi, y_test_multi = multivariate_data(test_set, test_y, 0,
                                               None, past_history,
                                               future_target,
                                               single_step=False)

In [ ]:
x_test_multi.shape

(363, 24, 15)

In [ ]:
# data object to float
x_train_multi = x_train_multi.astype(float)
x_val_multi = x_val_multi.astype(float)
x_test_multi = x_test_multi.astype(float)

y_train_multi = y_train_multi.astype(float)
y_val_multi = y_val_multi.astype(float)
y_test_multi = y_test_multi.astype(float)